In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical 
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import BatchNormalization
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Definitions

In [ ]:
WIDTH, HEIGHT = IMAGE_SIZE = (64, 64)
DEPTH = 3
DATASET_PATH = "dataset"

# Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(DATASET_PATH + "/train", target_size=IMAGE_SIZE, batch_size=32)

In [ ]:
class_indices = training_set.class_indices
train_labels = training_set.classes
train_labels = to_categorical(train_labels, num_classes=len(class_indices))
class_indices

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(DATASET_PATH + "/test",
                                            target_size = IMAGE_SIZE,
                                            batch_size = 32)

# Building the CNN


In [ ]:
cnn = Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu', input_shape=(HEIGHT, WIDTH, DEPTH)))
cnn.add(BatchNormalization(axis=-1))

In [ ]:
cnn.add(MaxPool2D(pool_size=(3, 3)))
cnn.add(Dropout(0.25))

### Step 2- Adding a second convolutional layer

In [ ]:
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
cnn.add(BatchNormalization(axis=-1))
cnn.add(MaxPool2D(pool_size=(3, 3)))
cnn.add(Dropout(0.2))

### Step 3 - Flattening

In [ ]:
cnn.add(Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(Dense(units=256, activation="relu"))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.15))

### Step 5 - Output Layer

In [ ]:
cnn.add(Dense(units=num_classes, activation='softmax'))

# Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# Saving the model

In [17]:
cnn.save(f'battery_analyser/model')